# ConvLSTM Additions for Wildfire Prediction (TensorFlow/Keras)

This add-on injects a ConvLSTM2D sequence model into your existing Keras workflow.

In [ ]:

import os, math, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
print('TensorFlow', tf.__version__)


In [ ]:

def make_sliding_windows(frames, T, stride=1):
    import numpy as np
    N = frames.shape[0]
    M = 1 + (N - T) // stride if N >= T else 0
    out = np.stack([frames[i:i+T] for i in range(0, N - T + 1, stride)], axis=0) if M > 0 else np.empty((0, T) + frames.shape[1:])
    return out

def make_tf_dataset(X_seq, y, batch_size=8, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X_seq, y))
    if shuffle:
        ds = ds.shuffle(min(len(X_seq), 2048), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:

from tensorflow.keras import layers, optimizers

def conv_block(x, f, k=3, s=1, p='same', act='relu', bn=True, drop=0.0):
    x = layers.Conv2D(f, k, strides=s, padding=p, use_bias=not bn)(x)
    if bn: x = layers.BatchNormalization()(x)
    x = layers.Activation(act)(x)
    if drop > 0: x = layers.Dropout(drop)(x)
    return x

def build_convlstm_model(input_shape, num_classes, 
                         convlstm_filters=(32, 64),
                         convlstm_kernel=3,
                         convlstm_drop=0.1,
                         post_cnn_filters=(64, 64),
                         gap_drop=0.2):
    inputs = keras.Input(shape=input_shape)
    x = layers.TimeDistributed(layers.Conv2D(32, 3, padding='same', use_bias=False))(inputs)
    x = layers.TimeDistributed(layers.BatchNormalization())(x)
    x = layers.TimeDistributed(layers.Activation('relu'))(x)
    for i, f in enumerate(convlstm_filters):
        return_seq = (i < len(convlstm_filters) - 1)
        x = layers.ConvLSTM2D(filters=f, kernel_size=convlstm_kernel, padding='same',
                              return_sequences=return_seq, dropout=convlstm_drop)(x)
        if return_seq:
            x = layers.TimeDistributed(layers.BatchNormalization())(x)
        else:
            x = layers.BatchNormalization()(x)
    for f in post_cnn_filters:
        x = conv_block(x, f, k=3, bn=True, drop=0.1)
    x = layers.GlobalAveragePooling2D()(x)
    if gap_drop > 0: x = layers.Dropout(gap_drop)(x)
    if num_classes == 1:
        outputs = layers.Dense(1, activation='sigmoid')(x)
        loss = 'binary_crossentropy'
        metrics = ['accuracy', keras.metrics.AUC(name='auc')]
    else:
        outputs = layers.Dense(num_classes, activation='softmax')(x)
        loss = 'sparse_categorical_crossentropy'
        metrics = ['accuracy', keras.metrics.AUC(name='auc', multi_label=False)]
    model = keras.Model(inputs, outputs, name='ConvLSTM_Wildfire')
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss=loss, metrics=metrics)
    return model


In [ ]:

# Example usage (replace with real data)
# X_train_seq = np.random.rand(32, 6, 64, 64, 6).astype('float32')
# y_train = np.random.randint(0, 2, size=(32,)).astype('int32')
# X_val_seq = np.random.rand(8, 6, 64, 64, 6).astype('float32')
# y_val = np.random.randint(0, 2, size=(8,)).astype('int32')
# ds_train = make_tf_dataset(X_train_seq, y_train, batch_size=4, shuffle=True)
# ds_val = make_tf_dataset(X_val_seq, y_val, batch_size=4, shuffle=False)
# model = build_convlstm_model(input_shape=X_train_seq.shape[1:], num_classes=2)
# ckpt = ModelCheckpoint('best_convlstm.keras', monitor='val_auc', mode='max', save_best_only=True, verbose=1)
# es = EarlyStopping(monitor='val_auc', mode='max', patience=8, restore_best_weights=True, verbose=1)
# history = model.fit(ds_train, validation_data=ds_val, epochs=50, callbacks=[ckpt, es])
# print('Best model saved to best_convlstm.keras')
